In [0]:
# Python 3, TensorFlow 1.0

import os
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


#########
# 옵션 설정
######
n_width = 28   # MNIST 이미지의 가로 크기, RNN의 input 갯수
n_height = 28  # MNIST 이미지의 세로 크기, RNN의 step 수
n_output = 10  # 0~9
learning_rate = 0.001

#########
# CNN
######
def CNN2(input_X):
    L1 = tf.contrib.layers.conv2d(input_X, 32, [3, 3],
                                  normalizer_fn=tf.nn.dropout,
                                  normalizer_params={'keep_prob': 0.8})
    L2 = tf.contrib.layers.max_pool2d(L1, [2, 2])
    L3 = tf.contrib.layers.conv2d(L2, 64, [3, 3],
                                  normalizer_fn=tf.nn.dropout,
                                  normalizer_params={'keep_prob': 0.8})
    L4 = tf.contrib.layers.max_pool2d(L3, [2, 2])

    L5 = tf.contrib.layers.flatten(L4)
    L5 = tf.contrib.layers.fully_connected(L5, 256,
                                           normalizer_fn=tf.contrib.layers.batch_norm)

    return tf.contrib.layers.fully_connected(L5, n_output)

def CNN(input_X, keep_prob, training):
    L1 = tf.layers.conv2d(input_X, 32, [3, 3], padding='same', activation=tf.nn.relu)
    L1 = tf.layers.max_pooling2d(L1, [2, 2], strides=2)

    L2 = tf.layers.conv2d(L1, 64, [3, 3], padding='same', activation=tf.nn.relu)
    L2 = tf.layers.max_pooling2d(L2, [2, 2], strides=2)
    L2 = tf.layers.dropout(L2, keep_prob, training)

    L3 = tf.layers.conv2d(L2, 128, [3, 3], padding='same', activation=tf.nn.relu)
    L3 = tf.layers.max_pooling2d(L3, [2, 2], strides=2)

    L3 = tf.contrib.layers.flatten(L3)
    L4 = tf.layers.dense(L3, n_height * n_width, activation=tf.nn.relu)
    L4 = tf.layers.dropout(L4, keep_prob, training)

    return tf.layers.dense(L4, n_output)

#########
# RNN
#####
def RNN(input_X, keep_prob, training):
    cell = tf.contrib.rnn.BasicLSTMCell(128)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * 2)

    outputs, states = tf.nn.dynamic_rnn(cell, input_X, dtype=tf.float32)

    outputs = tf.contrib.layers.flatten(outputs)
    outputs = tf.layers.dense(outputs, n_height * n_width, activation=tf.nn.relu)
    outputs = tf.layers.dropout(outputs, keep_prob, training)
    outputs = tf.layers.dense(outputs, n_output)

    return outputs







Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
RNN_model = RNN(RNN_X, keep_prob, is_training)

ValueError: ignored

In [0]:
global_step = tf.Variable(0, trainable=False, name="global_step")
is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)

CNN_X = tf.placeholder(tf.float32, [None, n_width, n_height, 1])
RNN_X = tf.placeholder(tf.float32, [None, n_height, n_width])
Y = tf.placeholder(tf.float32, [None, n_output])

CNN_model = CNN(CNN_X, keep_prob, is_training)
CNN_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=CNN_model, labels=Y))
CNN_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(CNN_cost, global_step=global_step)

RNN_model = RNN(RNN_X, keep_prob, is_training)
RNN_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=RNN_model, labels=Y))
RNN_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(RNN_cost, global_step=global_step)

ValueError: ignored

In [0]:
tf.__version__

'1.14.0'

In [0]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state("./model")
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())


batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(30):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        CNN_batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        RNN_batch_xs = batch_xs.reshape(-1, 28, 28)

        _, CNN_cost_val = sess.run([CNN_optimizer, CNN_cost],
                                   feed_dict={CNN_X: CNN_batch_xs, Y: batch_ys,
                                              keep_prob: 0.6, is_training: True})
        _, RNN_cost_val = sess.run([RNN_optimizer, RNN_cost],
                                   feed_dict={RNN_X: RNN_batch_xs, Y: batch_ys,
                                              keep_prob: 0.6, is_training: True})

        total_cost += CNN_cost_val + RNN_cost_val

    print('Epoch:', '%04d' % epoch, \
          'Avg. cost =', '{:.4f}'.format(total_cost / total_batch))

checkpoint_path = os.path.join("./model", "mnist.ckpt")
saver.save(sess, checkpoint_path)

print('최적화 완료!')


#########
# 결과 확인
######
CNN_is_correct = tf.equal(tf.argmax(CNN_model, 1), tf.argmax(Y, 1))
CNN_accuracy = tf.reduce_mean(tf.cast(CNN_is_correct, tf.float32))

RNN_is_correct = tf.equal(tf.argmax(RNN_model, 1), tf.argmax(Y, 1))
RNN_accuracy = tf.reduce_mean(tf.cast(RNN_is_correct, tf.float32))

is_correct = tf.equal(tf.argmax(tf.add(CNN_model, RNN_model), 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

acc_val, CNN_acc_val, RNN_acc_val = sess.run([accuracy, CNN_accuracy, RNN_accuracy],
                                             feed_dict={CNN_X: mnist.test.images.reshape(-1, 28, 28, 1),
                                                        RNN_X: mnist.test.images.reshape(-1, 28, 28),
                                                        Y: mnist.test.labels,
                                                        keep_prob: 1,
                                                        is_training: False})

print('정확도: %.4f (CNN: %.4f, RNN: %.4f)' % (acc_val, CNN_acc_val, RNN_acc_val))

NameError: ignored

In [0]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.nn import init


class ConvGRUCell(nn.Module):
    """
    Generate a convolutional GRU cell
    """

    def __init__(self, input_size, hidden_size, kernel_size):
        super().__init__()
        padding = kernel_size // 2
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reset_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.update_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.out_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)

        init.orthogonal(self.reset_gate.weight)
        init.orthogonal(self.update_gate.weight)
        init.orthogonal(self.out_gate.weight)
        init.constant(self.reset_gate.bias, 0.)
        init.constant(self.update_gate.bias, 0.)
        init.constant(self.out_gate.bias, 0.)


    def forward(self, input_, prev_state):

        # get batch and spatial sizes
        batch_size = input_.data.size()[0]
        spatial_size = input_.data.size()[2:]

        # generate empty prev_state, if None is provided
        if prev_state is None:
            state_size = [batch_size, self.hidden_size] + list(spatial_size)
            if torch.cuda.is_available():
                prev_state = torch.zeros(state_size).cuda()
            else:
                prev_state = torch.zeros(state_size)

        # data size is [batch, channel, height, width]
        stacked_inputs = torch.cat([input_, prev_state], dim=1)
        update = F.sigmoid(self.update_gate(stacked_inputs))
        reset = F.sigmoid(self.reset_gate(stacked_inputs))
        out_inputs = F.tanh(self.out_gate(torch.cat([input_, prev_state * reset], dim=1)))
        new_state = prev_state * (1 - update) + out_inputs * update

        return new_state


class ConvGRU(nn.Module):

    def __init__(self, input_size, hidden_sizes, kernel_sizes, n_layers):
        '''
        Generates a multi-layer convolutional GRU.
        Preserves spatial dimensions across cells, only altering depth.
        Parameters
        ----------
        input_size : integer. depth dimension of input tensors.
        hidden_sizes : integer or list. depth dimensions of hidden state.
            if integer, the same hidden size is used for all cells.
        kernel_sizes : integer or list. sizes of Conv2d gate kernels.
            if integer, the same kernel size is used for all cells.
        n_layers : integer. number of chained `ConvGRUCell`.
        '''

        super(ConvGRU, self).__init__()

        self.input_size = input_size

        if type(hidden_sizes) != list:
            self.hidden_sizes = [hidden_sizes]*n_layers
        else:
            assert len(hidden_sizes) == n_layers, '`hidden_sizes` must have the same length as n_layers'
            self.hidden_sizes = hidden_sizes
        if type(kernel_sizes) != list:
            self.kernel_sizes = [kernel_sizes]*n_layers
        else:
            assert len(kernel_sizes) == n_layers, '`kernel_sizes` must have the same length as n_layers'
            self.kernel_sizes = kernel_sizes

        self.n_layers = n_layers

        cells = []
        for i in range(self.n_layers):
            if i == 0:
                input_dim = self.input_size
            else:
                input_dim = self.hidden_sizes[i-1]

            cell = ConvGRUCell(input_dim, self.hidden_sizes[i], self.kernel_sizes[i])
            name = 'ConvGRUCell_' + str(i).zfill(2)

            setattr(self, name, cell)
            cells.append(getattr(self, name))

        self.cells = cells


    def forward(self, x, hidden=None):
        '''
        Parameters
        ----------
        x : 4D input tensor. (batch, channels, height, width).
        hidden : list of 4D hidden state representations. (batch, channels, height, width).
        Returns
        -------
        upd_hidden : 5D hidden representation. (layer, batch, channels, height, width).
        '''
        if not hidden:
            hidden = [None]*self.n_layers

        input_ = x

        upd_hidden = []

        for layer_idx in range(self.n_layers):
            cell = self.cells[layer_idx]
            cell_hidden = hidden[layer_idx]

            # pass through layer
            upd_cell_hidden = cell(input_, cell_hidden)
            upd_hidden.append(upd_cell_hidden)
            # update input_ to the last updated hidden layer for next pass
            input_ = upd_cell_hidden

        # retain tensors in list to allow different hidden sizes
        return upd_hidden

In [0]:
# from convgru import ConvGRU

# Generate a ConvGRU with 3 cells
# input_size and hidden_sizes reflect feature map depths.
# Height and Width are preserved by zero padding within the module.
model = ConvGRU(input_size=28*28, hidden_sizes=[32,64,64],
                  kernel_sizes=[3, 5, 3], n_layers=3)


# model = ConvGRU(input_size=28, hidden_sizes=[32,64],
#                   kernel_sizes=[3, 5], n_layers=2)


x = torch.FloatTensor(1,28*28,64,64)
output = model(x)

# output is a list of sequential hidden representation tensors
print(type(output)) # list

# final output size


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use to

<class 'list'>


In [0]:
output[0].shape

torch.Size([1, 32, 64, 64])